In [1]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet
from bokeh.plotting import figure
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(0)
Engine._list_cached_validations()
#mp._fetch_installed_base(); # refresh local installed fleet database

,1486152,1330175,1486174,1486176
Asset ID,159397,114517,159398,159399
Validation Engine,Forsa Hartmoor M02,CREYKE BECK M03,Forsa Hartmoor M04,Forsa Hartmoor M03
n,999,999,999,999
oph@start,54.0,0,44.0,51.0
serialNumber,1486152,1330175,1486174,1486176
starts@start,57.0,0,61.0,52.0
val start,2022-01-10 00:00:00,2017-12-05 00:00:00,2022-01-11 00:00:00,2022-01-11 00:00:00
source,from_MyPlant,from_MyPlant,from_MyPlant,from_MyPlant


In [2]:
def sfun(x):
    #return all([ ("CREYKE BECK" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Yukon Energy" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ ("Forsa Hartmoor" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

,0,1,2,3,4,5,6,7,8,9,10
Count_OpHour,1831.0,1843.0,1810.0,1816.0,1818.0,1832.0,1809.0,1797.0,1732.0,1711.0,1806.0
OperationalCondition,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available
shutdown_counter,793.0,754.0,735.0,738.0,748.0,756.0,773.0,743.0,729.0,698.0,741.0
startup_counter,793.0,752.0,735.0,738.0,746.0,754.0,773.0,743.0,729.0,696.0,736.0
id,159396,159397,159399,159398,159400,159401,159403,159402,159404,159405,159406
Engine Version,H12,H12,H12,H12,H12,H12,H12,H12,H12,H12,H12
Engine Type,624,624,624,624,624,624,624,624,624,624,624
Engine Series,6,6,6,6,6,6,6,6,6,6,6
Country,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB
Commissioning Date,2022-01-10,2022-01-10,2022-01-11,2022-01-11,2022-01-22,2022-01-10,2022-01-11,2022-01-11,2022-01-11,2022-01-22,2022-01-11


In [3]:
motor = fleet.iloc[2]
modes = ['undefined','OFF','MAN','AUTO']
success = ['success','failed','undefined']
e=Engine.from_fleet(mp,motor)
#pp_from='2022-03-28 06:00' # 1 Start
#pp_to='2022-03-28 08:14'
pp_from='2022-08-01'
pp_to='2023-03-01'
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('Forsa Hartmoor M03',
 ['undefined', 'OFF', 'MAN', 'AUTO'],
 ['success', 'failed', 'undefined'])

In [4]:
# fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
# fsm.run0(enforce=True, silent=False, debug=False)
# fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
# fsm.run2(silent=False)
# fsm.run4(silent=False)

In [5]:
#fsm.save_results('Forsa_Hartmoor_M3_debug.dfsm')
fsm = FSMOperator.load_results(mp, 'Forsa_Hartmoor_M3_debug.dfsm')

In [6]:
pp_from = pd.to_datetime(fsm.results['starts'][-3]['starttime'])
pp_to = date.today()

In [7]:
#pp_from='2023-03-28'
#pp_to='2023-03-29'
fsm2 = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm2.run0(enforce=True, silent=False, debug=False)
fsm2.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm2.run2(silent=False)
fsm2.run4(silent=False)

FSM4: 100%|████████████████████████████████| 59/59 [00:03<00:00, 17.33 starts/s]


In [8]:
print(f"FSM  {fsm.results['starts_counter']} {fsm.results['first_message']} {fsm.results['last_message']}")
print(f"FSM2 {fsm2.results['starts_counter']} {fsm2.results['first_message']} {fsm2.results['last_message']}")

FSM  321 2022-08-01 00:10:17.091000064 2023-02-28 23:16:09.358000128
FSM2 59 2023-02-22 17:30:09.100999936 2023-05-04 23:53:58.904999936


In [9]:
# pp_from='2022-08-01'
# pp_to='2023-05-04'
# fsm3 = FSMOperator(e, p_from=pp_from, p_to=pp_to)
# fsm3.run0(enforce=True, silent=False, debug=False)
# fsm3.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
# fsm3.run2(silent=False)
# fsm3.run4(silent=False)
# print(f"FSM3 {fsm3.results['starts_counter']} {fsm3.results['first_message']} {fsm3.results['last_message']}")

In [10]:
#fsm3.results
#4.5.2020 validated, Dieter

In [11]:
for k in fsm.results.keys():
    print(k)

sn
save_date
first_message
last_message
starts
starts_counter
stops
run2_content
run4_content
serviceselectortiming
oilpumptiming
stops_counter
run2_failed
runlog
runlogdetail
run4_failed
info


In [12]:
print(f"SN: {fsm.results['sn']} - {fsm2.results['sn']}")
print(f"starts: {len(fsm.results['starts'])} - {len(fsm2.results['starts'])}")
print(f"stops: {len(fsm.results['stops'])} - {len(fsm2.results['stops'])}")
print(f"messages: {fsm.results['first_message']} - {fsm2.results['first_message']}")
print(f"messages: {fsm.results['last_message']} - {fsm2.results['last_message']}")
print()
for s in fsm.results['starts'][-4:]:
    print(f"Start: no {s['no']} {s['starttime']} {s['mode']}")

print()
for i in range(1,10):
    if fsm.results['starts'][-i]['starttime'] == fsm2.results['starts'][0]['starttime']:
        break

s = fsm.results['starts'][-i]
print(f"{i}: Start: no {s['no']} {s['starttime']} {s['mode']}")

print()
for j, s in enumerate(fsm2.results['starts'][:4]):
    if j < i:
        print('( ', end='')
        print(f"Start: no {s['no']} {s['starttime']} {s['mode']}", end='')
        print(' )')
    else:
        print(f"Start: no {s['no']} {s['starttime']} {s['mode']}")

#---
print('-----')
for s in fsm.results['stops'][-4:]:
    print(f"Stop: no {s['no']} {s['starttime']} {s['mode']}")

print()
for i in range(1,10):
    if fsm.results['stops'][-i]['starttime'] == fsm2.results['stops'][0]['starttime']:
        break

s = fsm.results['stops'][-i]
print(f"{i}: Stop: no {s['no']} {s['starttime']} {s['mode']}")

print()
for j, s in enumerate(fsm2.results['stops'][:4]):
    if j < i:
        print('( ', end='')
        print(f"Stop: no {s['no']} {s['starttime']} {s['mode']}", end='')
        print(' )')
    else:
        print(f"Stop: no {s['no']} {s['starttime']} {s['mode']}")


SN: 1486176 - 1486176
starts: 321 - 59
stops: 322 - 60
messages: 2022-08-01 00:10:17.091000064 - 2023-02-22 17:30:09.100999936
messages: 2023-02-28 23:16:09.358000128 - 2023-05-04 23:53:58.904999936

Start: no 317 2023-02-21 09:05:45.899000064 AUTO
Start: no 318 2023-02-22 17:30:08.897999872 AUTO
Start: no 319 2023-02-28 06:18:41.121999872 AUTO
Start: no 320 2023-02-28 07:17:02.832000 AUTO

2: Start: no 319 2023-02-28 06:18:41.121999872 AUTO

( Start: no 0 2023-02-28 06:18:41.121999872 undefined )
( Start: no 1 2023-02-28 07:17:02.832000 undefined )
Start: no 2 2023-03-03 06:30:08.793999872 undefined
Start: no 3 2023-03-03 17:30:11.583000064 undefined
-----
Stop: no 318 2023-02-21 09:47:24.056000 AUTO
Stop: no 319 2023-02-22 18:05:23.758000128 AUTO
Stop: no 320 2023-02-28 06:20:30.286000128 AUTO
Stop: no 321 2023-02-28 09:11:23.204000 AUTO

9: Stop: no 313 2023-02-13 12:24:44.566000128 AUTO

( Stop: no 0 2023-02-22 17:30:09.100999936 undefined )
( Stop: no 1 2023-02-28 06:20:30.2860001

In [13]:
fsm.merge_results(fsm2)


***********************************
** Merging SN 1486176
***********************************

** Merging lastmessage from 2023-02-28 23:16:09.358000128
** Merging lastmessage to 2023-05-04 23:53:58.904999936
** Merging: Skipping first 2 Starts to align.

** Merging: 321 Starts in base file
** Merging Added: 57 Starts
** Merging starts_counter: 378

** Merging: 322 Stops in base file
** Merging Added: 60 Stops
** Merging stops_counter: 382

** Merging dict elements :run2_content run4_content serviceselectortiming oilpumptiming run2_failed run4_failed
** Mergine runlogdetail (*Bug startnumbers start at - 1 currently), last start 319
** Mergine runlogdetails done


In [14]:
#4.5.2020 validated, Dieter
for s in fsm.results['starts']:
    print(f"Start: no {s['no']} {s['starttime']} {s['mode']}")

Start: no 0 2022-08-01 05:05:06.687000064 undefined
Start: no 1 2022-08-01 09:52:06.816999936 undefined
Start: no 2 2022-08-01 16:35:07.070000128 undefined
Start: no 3 2022-08-01 19:05:07.646000128 undefined
Start: no 4 2022-08-03 14:35:08.896999936 undefined
Start: no 5 2022-08-03 20:31:09.940000 undefined
Start: no 6 2022-08-04 06:05:09.879000064 undefined
Start: no 7 2022-08-04 15:05:04.129999872 undefined
Start: no 8 2022-08-05 05:48:54.264000 undefined
Start: no 9 2022-08-05 19:35:05.072000 undefined
Start: no 10 2022-08-07 17:35:06.921999872 undefined
Start: no 11 2022-08-08 05:09:13.708000 undefined
Start: no 12 2022-08-08 15:05:08.518000128 undefined
Start: no 13 2022-08-09 05:05:07.811000064 undefined
Start: no 14 2022-08-09 15:34:07.935000064 undefined
Start: no 15 2022-08-10 05:05:09.751000064 undefined
Start: no 16 2022-08-10 15:04:08.942000128 undefined
Start: no 17 2022-08-11 05:35:04.143000064 undefined
Start: no 18 2022-08-11 14:04:04.216999936 undefined
Start: no 19 20